# Implementing [Schmid 2019] using SQLite

[Bradley P. Allen](https://www.bradleypallen.org), first version 2022-05-08, second version 2023-07-30

## Overview

This Python notebook gist demonstrates the use of a very simple implementation of a relational database approach to storing and querying labeled property graphs as described by Matthias Schmid of the University of Passau [[1]](https://dl.acm.org/doi/abs/10.1145/3366030.3366046).

## Preliminaries

### Python library imports

First, we import some Python libraries. We'll use ```pandas``` to print out query results in an attractive format, Simon Willison's ```sqlite_utils``` to make the task of manipulating the SQLite database easy, and ```sqlite_graph``` to provide utility functions to set up the adjacency tables for the graph store and perform neighborhood queries using CTEs.

In [3]:
import pandas as pd
from sqlite_utils import Database
from sqlite_graph import generate_adjacency_tables, in_neighborhood_cte, out_neighborhood_cte

### Demo database cleanup

We'll also delete and recreate the SQLite database file we'll be using, just to keep things simple.

In [4]:
!rm -f test_schmid.db

In [5]:
db = Database('test_schmid.db')

### A utility function for displaying result sets as Pandas DataFrames

Just a little thing to display results in a more readable fashion than simply printing rows out.

In [6]:
def as_dataframe(rows, columns=None):
    df = pd.DataFrame( rows, columns=columns )
    df.index = [''] * len(df)
    return df

## Loading the graph

We hew closely to the example in Schmid's paper, specifically the data shown in Tables 2, 3, and 4 of [[1]](https://dl.acm.org/doi/abs/10.1145/3366030.3366046).

### The vertex table

In [7]:
db['vertex'].insert_all([
    { "vid": 1, "attributes": { "lastName": "Mueller", "firstName": "David" } },
    { "vid": 2, "attributes": { "lastName": "Choi", "firstName": "Jae-Jin" } },
    { "vid": 3, "attributes": { "lastName": "Yamamoto", "firstName": "Akira" } },
    { "vid": 4, "attributes": { "lastName": "Silva", "firstName": "Ana" } },
    { "vid": 5, "attributes": { "lastName": "Poussin", "firstName": "Jacques" } },
    { "vid": 6, "attributes": { "lastName": "Professorson", "firstName": "Derek" } },
    { "vid": 7, "attributes": { "lastName": "Abadi", "firstName": "Madiha" } },
], pk='vid')

<Table vertex (vid, attributes)>

In [8]:
print(db['vertex'].schema)

CREATE TABLE [vertex] (
   [vid] INTEGER PRIMARY KEY,
   [attributes] TEXT
)


In [9]:
as_dataframe( [ row for row in db['vertex'].rows ] )

,vid,attributes
,1,"{""lastName"": ""Mueller"", ""firstName"": ""David""}"
,2,"{""lastName"": ""Choi"", ""firstName"": ""Jae-Jin""}"
,3,"{""lastName"": ""Yamamoto"", ""firstName"": ""Akira""}"
,4,"{""lastName"": ""Silva"", ""firstName"": ""Ana""}"
,5,"{""lastName"": ""Poussin"", ""firstName"": ""Jacques""}"
,6,"{""lastName"": ""Professorson"", ""firstName"": ""Der..."
,7,"{""lastName"": ""Abadi"", ""firstName"": ""Madiha""}"


### The edge table

In [10]:
db['edge'].insert_all([ 
    { "eid": 4, "sid": 1, "tid": 2, "label": "knows", "attributes": { "since": "2018-06-14" } },
    { "eid": 5, "sid": 1, "tid": 3, "label": "knows", "attributes": { "since": "2016-03-21" } },
    { "eid": 6, "sid": 2, "tid": 7, "label": "likes" },
], pk='eid')

<Table edge (eid, sid, tid, label, attributes)>

In [11]:
print(db['edge'].schema)

CREATE TABLE [edge] (
   [eid] INTEGER PRIMARY KEY,
   [sid] INTEGER,
   [tid] INTEGER,
   [label] TEXT,
   [attributes] TEXT
)


In [12]:
as_dataframe( [ row for row in db['edge'].rows ] )

,eid,sid,tid,label,attributes
,4,1,2,knows,"{""since"": ""2018-06-14""}"
,5,1,3,knows,"{""since"": ""2016-03-21""}"
,6,2,7,likes,None


### The adjacency tables

In [13]:
generate_adjacency_tables(db)

In [14]:
print(db['outgoing'].schema)

CREATE TABLE [outgoing] (
   [vid] INTEGER PRIMARY KEY,
   [label_54] TEXT,
   [edges_54] TEXT,
   [label_38] TEXT,
   [edges_38] TEXT
)


In [15]:
as_dataframe( [ row for row in db['outgoing'].rows ] )

,vid,label_54,edges_54,label_38,edges_38
,1,knows,"[{""eid"": 4, ""tid"": 2}, {""eid"": 5, ""tid"": 3}]",None,None
,2,None,None,likes,"[{""eid"": 6, ""tid"": 7}]"


In [16]:
print(db['incoming'].schema)

CREATE TABLE [incoming] (
   [vid] INTEGER PRIMARY KEY,
   [label_54] TEXT,
   [edges_54] TEXT,
   [label_38] TEXT,
   [edges_38] TEXT
)


In [17]:
as_dataframe( [ row for row in db['incoming'].rows ] )

,vid,label_54,edges_54,label_38,edges_38
,2,knows,"[{""eid"": 4, ""sid"": 1}]",None,None
,3,knows,"[{""eid"": 5, ""sid"": 1}]",None,None
,7,None,None,likes,"[{""eid"": 6, ""sid"": 2}]"


## Querying the graph

In [18]:
as_dataframe([ row for row in db.execute(out_neighborhood_cte(1, "knows")).fetchall() ],
             columns=['vid', 'eid', 'label', 'tid'] )

,vid,eid,label,tid
,1,4,knows,2
,1,5,knows,3


In [19]:
as_dataframe([ row for row in db.execute(out_neighborhood_cte(2, "likes")).fetchall() ],
             columns=['vid', 'eid', 'label', 'tid'] )

,vid,eid,label,tid
,2,6,likes,7


### Who does David Mueller know?

In [20]:
query_1 = (
    f'with unshred_edges as ( {out_neighborhood_cte(1, "knows")} ),'
    ' targets as ( select tid from unshred_edges )'
    ' select json_extract(attributes, "$.lastName"),'
    ' json_extract(attributes, "$.firstName")'
    ' from vertex, targets where vertex.vid = tid'
)

In [21]:
as_dataframe( [ row for row in db.execute(query_1).fetchall() ],
             columns=['lastName', 'firstName'] )

,lastName,firstName
,Choi,Jae-Jin
,Yamamoto,Akira


### Who do people David Mueller knows like? 

#### Using the edge table

In [22]:
query_2 = (
    f'select json_extract(vertex.attributes, "$.lastName"),'
    ' json_extract(vertex.attributes, "$.firstName")'
    ' from vertex, edge as e1, edge as e2 where'
    ' e1.sid = 1 and'
    ' e1.label = "knows" and'
    ' e2.sid = e1.tid and'
    ' e2.label = "likes" and'
    ' vertex.vid = e2.tid'
)

In [23]:
as_dataframe( [ row for row in db.execute(query_2).fetchall() ],
             columns=['lastName', 'firstName'] )

,lastName,firstName
,Abadi,Madiha


#### Using the outgoing (adjacency) table

In [35]:
query_3 = (
    f'with knows_edges as ( {out_neighborhood_cte(1, "knows")} ),'
    ' likes_edges as ('
    ' select outgoing.vid as vid,'
    ' json_extract(value, "$.eid") as eid,'
    ' label_38 as label, json_extract(value, "$.tid") as tid from knows_edges,'
    ' outgoing, json_each(outgoing.edges_38)'
    ' where outgoing.vid = knows_edges.tid and label_38 = "likes"),'
    ' targets as ( select tid from likes_edges )'
    ' select json_extract(attributes, "$.lastName"),'
    ' json_extract(attributes, "$.firstName")'
    ' from vertex, targets where vertex.vid = tid'
)

In [36]:
query_3

'with knows_edges as ( select vid, json_extract(value, "$.eid") as eid, label_54 as label, json_extract(value, "$.tid") as tid from outgoing, json_each(outgoing.edges_54) where vid = "1" and label_54 = "knows" ), likes_edges as ( select outgoing.vid as vid, json_extract(value, "$.eid") as eid, label_38 as label, json_extract(value, "$.tid") as tid from knows_edges, outgoing, json_each(outgoing.edges_38) where outgoing.vid = knows_edges.tid and label_38 = "likes"), targets as ( select tid from likes_edges ) select json_extract(attributes, "$.lastName"), json_extract(attributes, "$.firstName") from vertex, targets where vertex.vid = tid'

In [46]:
query_3_1 = (
    'with edges_0 as ( select outgoing.vid as vid, json_extract(value, "$.eid") as eid, label_54 as label, json_extract(value, "$.tid") as tid from outgoing, json_each(outgoing.edges_54) where outgoing.vid = "1" and label_54 = "knows" ),'
    ' edges_1 as ( select outgoing.vid as vid, json_extract(value, "$.eid") as eid, label_38 as label, json_extract(value, "$.tid") as tid from edges_0, outgoing, json_each(outgoing.edges_38) where outgoing.vid = edges_0.tid and label_38 = "likes"),'
    ' targets as ( select tid from edges_1 )'
    ' select json_extract(attributes, "$.lastName"), json_extract(attributes, "$.firstName") from vertex, targets'
    ' where vertex.vid = tid'
)

In [47]:
as_dataframe( [ row for row in db.execute(query_3_1).fetchall() ],
             columns=['lastName', 'firstName'] )

,lastName,firstName
,Abadi,Madiha


### Who has David Mueller known the longest?

In [26]:
query_4 = (
    f'with unshred_edges as ( {out_neighborhood_cte(1, "knows")} ),'
    ' targets as ( select eid, tid from unshred_edges )'
    ' select json_extract(vertex.attributes, "$.lastName"),'
    ' json_extract(vertex.attributes, "$.firstName"),'
    ' min(json_extract(edge.attributes, "$.since"))'
    ' from vertex, edge, targets where vertex.vid = targets.tid and edge.eid = targets.eid'
)

In [27]:
as_dataframe( [ row for row in db.execute(query_4).fetchall() ],
             columns=['lastName', 'firstName', 'since'] )

,lastName,firstName,since
,Yamamoto,Akira,2016-03-21


### How many "knows" edges are there?

In [28]:
query_5 = f'select count(eid) from edge where label = "knows"'

In [29]:
db.execute(query_5).fetchone()[0]

2

### How many "knows" edges were created since 2017?

In [30]:
query_6 = (
    f'select count(eid) '
    'from edge where label = "knows" and '
    'json_extract(edge.attributes, "$.since") > "2017"'
)

In [31]:
db.execute(query_6).fetchone()[0]

1

### How many vertices have last names that start with the letter "P"?

In [32]:
query_7 = f'select count(vid) from vertex where json_extract(vertex.attributes, "$.lastName") LIKE "P%"'

In [33]:
db.execute(query_7).fetchone()[0]

2